# Lab 4: Pretraga hiperparametara

## Zadatak: priprema i obrada Tips skupa podataka

1. **Učitavanje realnog skupa**: Učitajte taxis skup podataka upotrebom sns biblioteke: https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page
2. **Vektorizacija kategorickih podataka**: Identifikovati kategoričke kolone. Na odgovarajućim kolonama primjeniti LabelEncoder i OneHotEncoder
3. **Vremenski podaci** - Vektorizovati vremenske podatke na najbolji mogući način
4. **Imputacija**: Detektovati da li postoje NaN vrijednosti te ih rješiti odgovarajućom tehnikom. Detektovati kolone koje posjeduju visok stepen korelacije te izbrisati odgovarajuće kolone.
5. **Outlieri**: Definisati prag od 3 standardne devijacije za numeričke kolonu total_bill te izvršiti uklanjanje outlier-a.
6. **Skaliranje**: Primijeniti `StandardScaler` ili `MinMaxScaler` na total_bill koloni.
7. **Trening**: Podijeliti podatke na training, test i validacioni skup, te izvršiti pretragu hiperparametara. Koristiti decision tree model za regresiju. Ciljna kolona: tip.
8. **Evaluacija**: Kreirati finalni model sa najboljim parametrim i evaluirati metrike na testnom skupu podataka.

In [1]:
import seaborn as sns
import numpy as np
import pandas as pd

df = sns.load_dataset('taxis')
print(f'Originalni oblik: {df.shape}')
df.head()

Originalni oblik: (6433, 14)


,pickup,dropoff,passengers,distance,fare,tip,tolls,total,color,payment,pickup_zone,dropoff_zone,pickup_borough,dropoff_borough
0,2019-03-23 20:21:09,2019-03-23 20:27:24,1,1.60,7.0,2.15,0.0,12.95,yellow,credit card,Lenox Hill West,UN/Turtle Bay South,Manhattan,Manhattan
1,2019-03-04 16:11:55,2019-03-04 16:19:00,1,0.79,5.0,0.00,0.0,9.30,yellow,cash,Upper West Side South,Upper West Side South,Manhattan,Manhattan
2,2019-03-27 17:53:01,2019-03-27 18:00:25,1,1.37,7.5,2.36,0.0,14.16,yellow,credit card,Alphabet City,West Village,Manhattan,Manhattan
3,2019-03-10 01:23:59,2019-03-10 01:49:51,1,7.70,27.0,6.15,0.0,36.95,yellow,credit card,Hudson Sq,Yorkville West,Manhattan,Manhattan
4,2019-03-30 13:27:42,2019-03-30 13:37:14,3,2.16,9.0,1.10,0.0,13.40,yellow,credit card,Midtown East,Yorkville West,Manhattan,Manhattan


In [22]:
def process_time_duration(df):
    df = df.copy()
    df['duration'] = (df['dropoff'] - df['pickup']).dt.total_seconds() / 60
    return df


def process_time_binning_interval(df, interval=30):
    df = df.copy()
    mins = df['pickup'].dt.hour * 60 + df['pickup'].dt.minute
    bins = np.arange(0, 1440 + interval, interval)
    labels = [f"{b//60:02d}:{b%60:02d}" for b in bins[1:]]
    df['time_bin'] = pd.cut(mins, bins=bins, labels=labels, right=False) # time_bin je categorical type
    return df


def process_time_cyclical(df):
    df = df.copy()
    h = df['pickup'].dt.hour
    df['hour_sin'] = np.sin(2*np.pi*h/24)
    df['hour_cos'] = np.cos(2*np.pi*h/24)
    return df

def process_time_month_seasons(df):
    """Map month to season: Winter, Spring, Summer, Fall."""
    df = df.copy()
    month = df['pickup'].dt.month
    bins = [0, 2, 5, 8, 11, 12]
    labels = ['Winter', 'Spring', 'Summer', 'Fall', 'Winter']
    df['season'] = pd.cut(month, bins=bins, labels=labels, right=True, ordered=False)
    return df


def process_time_month_categorical(df):
    """Use each month as a categorical variable."""
    df = df.copy()
    df['month'] = df['pickup'].dt.month_name()
    return df


def process_time_dayofweek(df):
    """Categorize day of week (Monday ... Sunday)."""
    df = df.copy()
    df['day_of_week'] = df['pickup'].dt.day_name()
    return df


def process_time_weekday_weekend(df):
    """Binary feature: Weekday vs Weekend."""
    df = df.copy()
    df['is_weekend'] = df['pickup'].dt.dayofweek >= 5
    return df


def process_time_hour_slots(df):
    """Bin hours into Night/Morning/WorkHours/Evening."""
    df = df.copy()
    hr = df['pickup'].dt.hour
    conditions = [hr.between(0,5), hr.between(6,11), hr.between(12,17), hr.between(18,23)]
    labels = ['Night', 'Morning', 'WorkHours', 'Evening']
    df['hour_slot'] = np.select(conditions, labels, default='Unknown')
    return df

# Moguc jos veliki broj obrada vremena

approaches = {
    'duration': process_time_duration,
    'bin_30min': lambda df: process_time_binning_interval(df, 30),
    'cyclical': process_time_cyclical,
    'season': process_time_month_seasons,
    'month_cat': process_time_month_categorical,
    'dayofweek': process_time_dayofweek,
    'weekend_flag': process_time_weekday_weekend,
    'hour_slots': process_time_hour_slots
}

In [23]:
# Mozemo kombinovati razlicite pristupe da dobijemo finalni rezultat
df_time_encoded = approaches['duration'](df)
df_time_encoded = approaches['season'](df_time_encoded)
df_time_encoded = approaches['weekend_flag'](df_time_encoded)
df_time_encoded = approaches['hour_slots'](df_time_encoded)
df_time_encoded = df_time_encoded.drop(columns=['pickup', 'dropoff'])
df_time_encoded.head()

,passengers,distance,fare,tip,tolls,total,color,payment,pickup_zone,dropoff_zone,pickup_borough,dropoff_borough,duration,season,is_weekend,hour_slot
0,1,1.60,7.0,2.15,0.0,12.95,yellow,credit card,Lenox Hill West,UN/Turtle Bay South,Manhattan,Manhattan,6.250000,Spring,True,Evening
1,1,0.79,5.0,0.00,0.0,9.30,yellow,cash,Upper West Side South,Upper West Side South,Manhattan,Manhattan,7.083333,Spring,False,WorkHours
2,1,1.37,7.5,2.36,0.0,14.16,yellow,credit card,Alphabet City,West Village,Manhattan,Manhattan,7.400000,Spring,False,WorkHours
3,1,7.70,27.0,6.15,0.0,36.95,yellow,credit card,Hudson Sq,Yorkville West,Manhattan,Manhattan,25.866667,Spring,True,Night
4,3,2.16,9.0,1.10,0.0,13.40,yellow,credit card,Midtown East,Yorkville West,Manhattan,Manhattan,9.533333,Spring,True,WorkHours


U zavisnosti od odabranog načina obrade vremena nastavljamo sa daljom obradom kategoričkih i numeričkih podataka

In [24]:
df_time_encoded.isnull().sum()

passengers          0
distance            0
fare                0
tip                 0
tolls               0
total               0
color               0
payment            44
pickup_zone        26
dropoff_zone       45
pickup_borough     26
dropoff_borough    45
duration            0
season              0
is_weekend          0
hour_slot           0
dtype: int64

In [27]:
from sklearn.preprocessing import LabelEncoder

df_encoded = df_time_encoded.copy()

categorical_columns = ['is_weekend']
label_encoders = {}

for column in categorical_columns:
    le = LabelEncoder()
    df_encoded[column] = le.fit_transform(df_time_encoded[column])
    label_encoders[column] = le

df_encoded.head()

,passengers,distance,fare,tip,tolls,total,color,payment,pickup_zone,dropoff_zone,pickup_borough,dropoff_borough,duration,season,is_weekend,hour_slot
0,1,1.60,7.0,2.15,0.0,12.95,yellow,credit card,Lenox Hill West,UN/Turtle Bay South,Manhattan,Manhattan,6.250000,Spring,1,Evening
1,1,0.79,5.0,0.00,0.0,9.30,yellow,cash,Upper West Side South,Upper West Side South,Manhattan,Manhattan,7.083333,Spring,0,WorkHours
2,1,1.37,7.5,2.36,0.0,14.16,yellow,credit card,Alphabet City,West Village,Manhattan,Manhattan,7.400000,Spring,0,WorkHours
3,1,7.70,27.0,6.15,0.0,36.95,yellow,credit card,Hudson Sq,Yorkville West,Manhattan,Manhattan,25.866667,Spring,1,Night
4,3,2.16,9.0,1.10,0.0,13.40,yellow,credit card,Midtown East,Yorkville West,Manhattan,Manhattan,9.533333,Spring,1,WorkHours


In [28]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

categorical_ohe_columns = ['color', 'payment', 'pickup_zone', 'dropoff_zone', 'pickup_borough', 'dropoff_borough', 'season', 'hour_slot']
ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
encoded_data = ohe.fit_transform(df_encoded[categorical_ohe_columns])
feature_names = ohe.get_feature_names_out(categorical_ohe_columns)
df_ohe = pd.DataFrame(encoded_data, columns=feature_names, index=df.index)
df_ohe.head()

,color_green,color_yellow,payment_cash,payment_credit card,payment_nan,pickup_zone_Allerton/Pelham Gardens,pickup_zone_Alphabet City,pickup_zone_Astoria,pickup_zone_Auburndale,pickup_zone_Battery Park,...,dropoff_borough_Manhattan,dropoff_borough_Queens,dropoff_borough_Staten Island,dropoff_borough_nan,season_Spring,season_Winter,hour_slot_Evening,hour_slot_Morning,hour_slot_Night,hour_slot_WorkHours
0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
1,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
2,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
3,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


In [29]:
df_encoded = df_encoded.drop(columns=categorical_ohe_columns).join(df_ohe)
df_encoded.head()

,passengers,distance,fare,tip,tolls,total,duration,is_weekend,color_green,color_yellow,...,dropoff_borough_Manhattan,dropoff_borough_Queens,dropoff_borough_Staten Island,dropoff_borough_nan,season_Spring,season_Winter,hour_slot_Evening,hour_slot_Morning,hour_slot_Night,hour_slot_WorkHours
0,1,1.60,7.0,2.15,0.0,12.95,6.250000,1,0.0,1.0,...,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
1,1,0.79,5.0,0.00,0.0,9.30,7.083333,0,0.0,1.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
2,1,1.37,7.5,2.36,0.0,14.16,7.400000,0,0.0,1.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
3,1,7.70,27.0,6.15,0.0,36.95,25.866667,1,0.0,1.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,3,2.16,9.0,1.10,0.0,13.40,9.533333,1,0.0,1.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


In [30]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scale_cols = ['distance', 'fare', 'duration', 'total']
df_encoded[scale_cols] = scaler.fit_transform(df_encoded[scale_cols])
df_encoded.head()

,passengers,distance,fare,tip,tolls,total,duration,is_weekend,color_green,color_yellow,...,dropoff_borough_Manhattan,dropoff_borough_Queens,dropoff_borough_Staten Island,dropoff_borough_nan,season_Spring,season_Winter,hour_slot_Evening,hour_slot_Morning,hour_slot_Night,hour_slot_WorkHours
0,1,-0.372199,-0.527324,2.15,0.0,-0.403040,-0.695665,1,0.0,1.0,...,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
1,1,-0.583821,-0.700471,0.00,0.0,-0.667255,-0.624091,0,0.0,1.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
2,1,-0.432289,-0.484038,2.36,0.0,-0.315451,-0.596893,0,0.0,1.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
3,1,1.221502,1.204142,6.15,0.0,1.334266,0.989183,1,0.0,1.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,3,-0.225892,-0.354178,1.10,0.0,-0.370465,-0.413664,1,0.0,1.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


In [31]:
y = df_encoded['tip'].to_numpy()
X = df_encoded.drop(columns=['tip']).to_numpy()

In [32]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [43]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import mean_absolute_error

param_grid = {
    'criterion': ['squared_error', 'friedman_mse', 'poisson'],
    'max_depth': [None, 5, 10],
    'min_samples_leaf': [1, 2, 4, 8],
    'min_samples_split': [2, 4, 8],
}

manual_grid = ParameterGrid(param_grid)

best_score = np.inf
best_params = None

for params in manual_grid:
    clf = DecisionTreeRegressor(**params)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_val)
    score = mean_absolute_error(y_val, y_pred)
    if score < best_score:
        best_score, best_params = score, params

print("Best score:", best_score)
print("Best parameters:", best_params)

Best score: 0.27446601941747584
Best parameters: {'criterion': 'squared_error', 'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2}


In [44]:
clf = DecisionTreeRegressor(**best_params)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
score = mean_absolute_error(y_test, y_pred)
print(f"MAE: {score}")

MAE: 0.33578088578088583
